In [ ]:
from gnutools import fs
from bpd.dask import DataFrame
from bpd.dask import functions as F
from gnutools.remote import gdrivezip

In [ ]:
# Define a function to be applied to each row
def word(f):
    return fs.name(fs.parent(f))

In [ ]:
# Import a sample dataset
gdrivezip("gdrive://1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE")
df = DataFrame({"filename": fs.listfiles("/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE", [".wav"])})

In [ ]:
# Chain transformations
df = df\
    .withColumn("name", F.apply(fs.name, F.col("filename")))\
    .withColumn("word", F.apply(word, F.col("filename")))\
    .set_index("word")

In [ ]:
#Display the dataframe
df.display()

In [ ]:
# Define a function to be applied to a groupby
def first(L):
    return L[0]

In [ ]:
# Join to the dataframe grouped by word. 
# The aggregation with generate a list of files for each unique word.
# Add a column `first` that will contain the result of the function first given the input of the filename column
# Drop the colummn filename 
# Join on word, right
# Reset the index and display
df.join(df\
        .select(["filename"])\
        .aggregate("word")\
        .withColumn("first", F.apply(first, F.col("filename")))\
        .drop_column(F.col("filename")), on="word", how="right"
       )\
.reset_index().display()